# Backward induction

In [1]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np

# c. local modules
from backward_induction import backward_induction_class

## Simulate data

In [2]:
# define model from class
model = backward_induction_class()

# expected value 0 
ev0 = np.zeros(model.n)






In [3]:
# simulate data (pk skal enten være 1 eller 0)
simdata = model.sim_data()


TypeError: sim_data() missing 2 required positional arguments: 'bellman' and 'ev0'